In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
cik = '796343'
def download_txt(cik):
    base = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb=&owner=include&count=100&search_text='.format(cik, '8-k')
    heads = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

    content = requests.get(base, headers = heads)
    bs4_base = BeautifulSoup(content.text)
    years = 0

    path_dict = {}
    
    for hrefs in bs4_base.find_all('a', href=True):

        if hrefs['href'].split('.')[-1] == 'htm' and len(hrefs['href'].split('/')) == 7:
            if hrefs.parent.parent.td.get_text().lower() != '8-k':
                continue
            years += 1
            htm_url = 'https://www.sec.gov/' + hrefs['href']
            content = requests.get(htm_url, headers = heads)
            bs4_htm = BeautifulSoup(content.text)

            for htm_hrefs in bs4_htm.find_all('a', href=True):
                if htm_hrefs['href'].split('.')[-1] == 'xml':
                    x = htm_hrefs['href'].split('.')[-2].split('_')[-1]
                    if x == 'def':
                        y = htm_hrefs['href'].split('/')[-1]
                        z = y.split('-')[1][:8]
                        break

            for htm_hrefs in bs4_htm.find_all('a', href=True):
                if htm_hrefs['href'].split('.')[-1] == 'txt':

                    url = 'https://www.sec.gov/' + htm_hrefs['href']
                    bs4_end = BeautifulSoup(requests.get(url, headers = heads).text)
                    y = url.split('/')[-1]
                    
                    file_path = '8k-forms/' + cik + '.txt'
                    path_dict[cik] = file_path
                    with open(file_path, "w", encoding="utf-8") as file:
                        file.write(str(bs4_end))

        if years >= 1:
            break

    return file_path


In [ ]:
df = pd.read_csv('base_comp_cik2.csv')
df = df[df['CIK'].isna() == False]

In [ ]:
all_paths = {}
for index, i in enumerate(df.index):
    cik = str(int(df.iloc[i]['CIK']))
    try:
        path = download_txt(cik)
        all_paths[cik] = path
    except:
        pass
    print(index, i)
        

In [ ]:
all_paths

In [ ]:
!conda install ipykernel --update-deps --force-reinstall